<a href="https://colab.research.google.com/github/Duarbert/SAD_DuarteSantos/blob/master/TPC5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
import io

# Importar os classificadores
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [0]:
data = pd.read_csv('/content/gdrive/My Drive/SAD/IBMHR.csv', index_col = 0)

In [0]:
pd.options.display.max_columns = None
#display(data)
data.columns = data.columns.str.strip()
data.columns



In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1470 entries, 41 to 34
Data columns (total 34 columns):
Attrition                   1470 non-null object
BusinessTravel              1470 non-null object
DailyRate                   1470 non-null int64
Department                  1470 non-null object
DistanceFromHome            1470 non-null int64
Education                   1470 non-null int64
EducationField              1470 non-null object
EmployeeCount               1470 non-null int64
EmployeeNumber              1470 non-null int64
EnvironmentSatisfaction     1470 non-null int64
Gender                      1470 non-null object
HourlyRate                  1470 non-null int64
JobInvolvement              1470 non-null int64
JobLevel                    1470 non-null int64
JobRole                     1470 non-null object
JobSatisfaction             1470 non-null int64
MaritalStatus               1470 non-null object
MonthlyIncome               1470 non-null int64
MonthlyRate            

In [0]:
def encode_target(df, target_column):
   df_mod = df.copy()
   targets = df_mod[target_column].unique()
   map_to_int = {name: n for n, name in enumerate(targets)}
   df_mod["Target"] = df_mod[target_column].replace(map_to_int)

   return (df_mod, targets)

In [0]:
df2, targets = encode_target(data, "Attrition")
print("* df2.head()", df2[["Target", "Attrition"]].head(),
      sep="\n", end="\n\n")
print("* df2.tail()", df2[["Target", "Attrition"]].tail(),
      sep="\n", end="\n\n")
print("* targets", targets, sep="\n", end="\n\n")

* df2.head()
     Target Attrition
Age                  
41        0       Yes
49        1        No
37        0       Yes
33        1        No
27        1        No

* df2.tail()
     Target Attrition
Age                  
36        1        No
39        1        No
27        1        No
49        1        No
34        1        No

* targets
['Yes' 'No']



In [0]:
y = df2['Attrition']

features = list(df2.columns[30:32])

x = df2[features]

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state = 5)

parameters = {'criterion':('gini', 'entropy'), 
              'min_samples_split':[2,3,4,5], 
              'max_depth':[9,10,11,12],
              'class_weight':('balanced', None),
              'presort':(False,True),
             }


tr = DecisionTreeClassifier()
gsearch = GridSearchCV(tr, parameters)
gsearch.fit(x_train, y_train)
model = gsearch.best_estimator_
model


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=9,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')